# 02. Fourier Transform and Time-Frequency analysis

In this notebook, we will not explain the exact mathematical formulation of the Fourier Transform or its theoretical derivation. 
Instead, we will focus on its practical application in the context of neural oscillation analysis and explore how it can be a powerful tool for understanding the frequency components of neural signals.

For those who are curious about the mathematical and intuitive aspects of the Fourier Transform, here is a fantastic video that provides a beautiful and intuitive visualization: 
[What is a Fourier Transform?](https://www.youtube.com/watch?v=spUNpyF58BY)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# I. The Fourier Transform

## I.0 Generate a sine wave

In [ ]:
# Define Sampling parameters
fs = 1000  # Sampling rate (1000 Hz)
duration = 4.0  # Seconds
t = np.arange(0, duration, 1/fs)

freq_1 = 6  # Frequency of the sine wave (6 Hz)
sig_1 = np.sin(2 * np.pi * freq_1 * t)  # Generate sine wave

In [ ]:
# Plot
plt.figure(figsize=(12, 4))
plt.plot(t, sig_1)
plt.title(f"Simulated Neural Signal at {freq_1} Hz")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")
plt.show()

## I.I Computing the Fourier Transform of the Sine Wave

In [ ]:
from scipy.fft import rfft, rfftfreq

# Calculate FFT
N = len(sig_1)  # Window length
yf_1 = rfft(sig_1)  # Compute the FFT
xf = rfftfreq(N, 1/fs) # Compute the frequency bins

In [ ]:
# Plot Power Spectrum
plt.figure(figsize=(10, 5))
plt.plot(xf, np.abs(yf_1))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)
plt.show()

The Fourier Transform is computed up to the Nyquist frequency of the signal, which is 500 Hz (half the sampling rate of 1000 Hz). We can now zoom in on the frequency of interest, 6 Hz.

In [ ]:
# Zoom in
plt.figure(figsize=(10, 5))
plt.plot(xf, np.abs(yf_1))
plt.xlim(0,15)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)
plt.show()

Let's try with another Sine Wave with a frequency of 15 Hz

In [ ]:
# Create a continuous oscillation (e.g., Alpha 15Hz)
freq_2 = 15
sig_2 = np.sin(2 * np.pi * freq_2 * t)

# Calculate FFT
yf_2 = rfft(sig_2)

In [ ]:
# Plot
plt.figure(figsize=(18, 4))

plt.subplot(1,2,1)
plt.plot(t, sig_2)
plt.title(f"Simulated Neural Signal at {freq_2} Hz")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")

plt.subplot(1,2,2)
plt.plot(xf, np.abs(yf_2))
plt.xlim(0,20)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)

In [ ]:
# Combine signals
sig_combined = sig_1 + sig_2
yf_combined = rfft(sig_combined)

In [ ]:
# Plot
plt.figure(figsize=(18, 4))

plt.subplot(1,2,1)
plt.plot(t, sig_combined, label='Raw LFP')
plt.title(f"Simulated Neural Signal")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")

plt.subplot(1,2,2)
plt.plot(xf, np.abs(yf_combined))
plt.xlim(0,20)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)

## I.2 Example with real data

In [ ]:
# Load recording
rec_50hz = np.load("data/50hz_traces.npz")
traces_50hz = rec_50hz["traces"][0,:] # Import first trace
fs_50hz = rec_50hz["fs"]    # Sampling rate

In [ ]:
# Calculate FFT
N_50z = len(traces_50hz)    # Window length
yf_50hz = rfft(traces_50hz)     # Compute the FFT
xf_50hz = rfftfreq(N_50z, 1/fs_50hz)    # Compute the frequency bins

duration_50hz = len(traces_50hz)/fs_50hz
t_50hz = np.arange(0, duration_50hz, 1/fs_50hz)

In [ ]:
# Plot
plt.figure(figsize=(18, 4))

plt.subplot(1,2,1)
plt.plot(t_50hz, traces_50hz)
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")

plt.subplot(1,2,2)
plt.plot(xf_50hz, np.abs(yf_50hz))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)

As before this signal contains a strong peak at 50Hz, most likely electric noise. We can apply the notch filter as before and see how it will be removed from the Power Spectrum

In [ ]:
# Notch filter
nyq = 0.5 * fs_50hz
order = 4
notch_cutoffs = [48 / nyq, 52 / nyq]
b, a = signal.butter(order, notch_cutoffs, btype='bandstop')
filtered_50Hz = signal.filtfilt(b, a, traces_50hz)

In [ ]:
# Calculate FFT
yf_filtered_50hz = rfft(filtered_50Hz)

In [ ]:
# Plot
plt.figure(figsize=(18, 4))

plt.subplot(1,2,1)
plt.plot(t_50hz, filtered_50Hz)
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")

plt.subplot(1,2,2)
plt.plot(xf_50hz, np.abs(yf_filtered_50hz))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)

# II. From FFT to Spectrogram: Time-Frequency Analysis

The Fourier Transform is great for identifying which frequencies are present in a signal, but it doesn't tell us *when* they occur. For a signal where the frequency content changes over time, we need a method that provides both time and frequency information.

This is where the **Short-Time Fourier Transform (STFT)** comes in. The idea is to break the signal into small, overlapping windows and compute the FFT for each window. By plotting these FFTs over time, we create a **spectrogram**.
 

## II.0 Create a signal with changing frequency

In [ ]:
t_part = np.arange(0, 2.0, 1/fs)
sig_part1 = np.sin(2 * np.pi * 6 * t_part)  # 6 Hz for first 2s
sig_part2 = np.sin(2 * np.pi * 15 * t_part) # 15 Hz for next 2s
sig_changing = np.concatenate([sig_part1, sig_part2])   # Concatenate parts

# Plot the signal
plt.figure(figsize=(12, 4))
plt.plot(t, sig_changing)
plt.title("Simulated Signal with Changing Frequency")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")
plt.show()

If we take the FFT of the entire signal, we see both frequency components, but we lose the information about when each one was active.

In [ ]:
# FFT of the whole signal
yf_changing = rfft(sig_changing)
xf_changing = rfftfreq(len(sig_changing), 1/fs)

plt.figure(figsize=(10, 5))
plt.plot(xf_changing, np.abs(yf_changing))
plt.xlim(0, 20)
plt.title("FFT of the Entire Signal")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)
plt.show()

## II.1 Building a Spectrogram Manually
Let's take two windows from the signal and compute their FFTs. One window from the first half, and one from the second half.
  

In [ ]:
# Define window parameters
win_len_sec = 1.0  # 1-second window
win_len_samples = int(win_len_sec * fs)

# Window 1: Centered at 1s
win1_start = int(0.5 * fs)
win1 = sig_changing[win1_start : win1_start + win_len_samples]
yf_win1 = rfft(win1)
xf_win = rfftfreq(len(win1), 1/fs)

# Window 2: Centered at 3s
win2_start = int(2.5 * fs)
win2 = sig_changing[win2_start : win2_start + win_len_samples]
yf_win2 = rfft(win2)

In [ ]:
# Plot the signal
plt.figure(figsize=(12, 4))
plt.plot(t, sig_changing)
plt.axvspan(t[win1_start], t[win1_start + win_len_samples], color='red', alpha=0.3, label="Window 1")
plt.axvspan(t[win2_start], t[win2_start + win_len_samples], color='green', alpha=0.3, label="Window 2")
plt.title("Simulated Signal with Changing Frequency")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")
plt.show()

# Plot the FFTs of the two windows
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4), sharey=True)
ax1.plot(xf_win, np.abs(yf_win1), "red")
ax1.set_title("FFT of Window at 1s")
ax1.set_xlabel("Frequency (Hz)")
ax1.set_ylabel("Power")
ax1.set_xlim(0, 20)
ax1.grid(True)

ax2.plot(xf_win, np.abs(yf_win2), "green")
ax2.set_title("FFT of Window at 3s")
ax2.set_xlabel("Frequency (Hz)")
ax2.set_xlim(0, 20)
ax2.grid(True)
plt.show()

As you can see, the FFT of the first window shows a peak at 6 Hz, and the second window shows a peak at 15 Hz. A spectrogram is simply the result of doing this for many windows and stacking the results."

In [ ]:
# Stack the magnitudes of yf_win1 and yf_win2
spectrogram_data = np.array([np.abs(yf_win1), np.abs(yf_win2)])

# Create a time-frequency representation
plt.figure(figsize=(12, 6))
plt.pcolormesh([0, 1], xf_win, spectrogram_data.T, shading='auto', cmap='jet')
plt.ylim(0,20)
plt.colorbar(label='Power')
plt.xticks([0, 1], ['Window 1', 'Window 2'])
plt.xlabel('Window Index')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram from Manually Computed FFTs')
plt.show()

## II.3 Generating a Spectrogram with SciPy
We can use `scipy.signal.spectrogram` to do this automatically.

In [ ]:
# Generate the spectrogram
f, t_spec, Sxx = signal.spectrogram(sig_changing, fs, nperseg=1000, noverlap=0)

# Plot the spectrogram
plt.figure(figsize=(12, 6))
plt.pcolormesh(t_spec, f, Sxx, shading='auto', cmap="jet")
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.ylim(0, 20)
plt.title('Spectrogram')
plt.colorbar(label='Power/Frequency')
plt.show()

# III. Spectrogram Parameters

## III.1 `nperseg`: The Time-Frequency Trade-off

A spectrogram's appearance and the information it conveys are highly dependent on its parameters. The most critical parameter is the **window length**, which controls the fundamental trade-off between time and frequency resolution.

In `scipy.signal.spectrogram`, this is controlled by `nperseg` (number of samples per segment).

- **Long Window (High `nperseg`)**: A longer window captures more of the signal for each FFT calculation. This provides a more detailed and accurate measurement of the frequencies present within that window, resulting in **high frequency resolution**. However, because the window is long, it's harder to pinpoint exactly *when* a frequency event occurred, leading to **low time resolution**.

- **Short Window (Low `nperseg`)**: A shorter window uses fewer samples. This allows you to localize frequency events very precisely in time, giving **high time resolution**. The drawback is that with fewer samples, the FFT has less data to work with, resulting in a coarser, less precise frequency measurement, or **low frequency resolution**.

This is known as the **Heisenberg-Gabor uncertainty principle**: you cannot simultaneously have perfect time and frequency resolution. You must choose which one is more important for your analysis.

Here is a great visual explanation of this principle in the context of Fourer transforms: [The more general uncertainty principle, regarding Fourier transforms](https://www.youtube.com/watch?v=MBnnXbOM5S4&t=422s)
  

In [ ]:
# Demonstrate the time-frequency trade-off

# 1. Long window: Good frequency resolution, poor time resolution
f_long, t_long, Sxx_long = signal.spectrogram(sig_changing, fs, nperseg=1200, noverlap=0)

# 2. Short window: Good time resolution, poor frequency resolution
f_short, t_short, Sxx_short = signal.spectrogram(sig_changing, fs, nperseg=200, noverlap=0)

# Calculate window borders for long and short windows (for plotting)
window_length_long = 1200 / fs  # in seconds
window_length_short = 200 / fs  # in seconds

borders_long = np.concatenate(([t_long[0] - window_length_long / 2], t_long + window_length_long / 2))
borders_short = np.concatenate(([t_short[0] - window_length_short / 2], t_short + window_length_short / 2))

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Long window spectrogram
ax1.pcolormesh(t_long, f_long, Sxx_long, shading='auto', cmap="jet")
ax1.set_title('Long Window (High Freq Resolution)')
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('Time [sec]')
ax1.set_ylim(0, 20)

# Add vertical lines for window borders in the long window
for border in borders_long:
    ax1.axvline(border, color='white', linestyle='--', alpha=0.7)

# Short window spectrogram
ax2.pcolormesh(t_short, f_short, Sxx_short, shading='auto', cmap="jet")
ax2.set_title('Short Window (High Time Resolution)')
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time [sec]')
ax2.set_ylim(0, 20)

# Add vertical lines for window borders in the short window
for border in borders_short:
    ax2.axvline(border, color='white', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## III.2 The Problem with Slow Frequencies

To accurately measure a frequency, the time window must be long enough to contain at least one full cycle of that frequency's wave (and ideally several).

For example, to resolve a slow 2 Hz frequency, you need a window of at least 1/2 = 0.5 seconds. If your window is shorter than that (e.g., 0.2 seconds), the analysis won't be able to "see" the full oscillation, and the power of that frequency will be smeared or missed entirely. This is a critical consideration when analyzing neural data, which often contains important slow oscillations like Delta (1-4 Hz).

In [ ]:
# Create a signal with changing frequency content
t = np.arange(0, 8.0, 1/fs)
t_part = np.arange(0, 4.0, 1/fs)
sig_part1 = np.sin(2 * np.pi * 2 * t_part)  # 2 Hz for first 2s
sig_part2 = np.sin(2 * np.pi * 8 * t_part) # 8 Hz for next 2s
sig_changing = np.concatenate([sig_part1, sig_part2])

# Plot the signal
plt.figure(figsize=(12, 4))
plt.plot(t, sig_changing)
plt.title("Simulated Signal with Changing Frequency")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (uV)")
plt.show()

In [ ]:
# Generate the spectrogram for window_length = 0.2s
window_length_1 = 0.2
overlapratio_1 = 0.0
nperseg_1 = int(window_length_1 * fs)
noverlap_1 = int(overlapratio_1 * nperseg_1)
f_1, t_spec_1, Sxx_1 = signal.spectrogram(sig_changing, fs, nperseg=nperseg_1, noverlap=noverlap_1)

# Generate the spectrogram for window_length = 1.0s
window_length_2 = 1.0
overlapratio_2 = 0.0
nperseg_2 = int(window_length_2 * fs)
noverlap_2 = int(overlapratio_2 * nperseg_2)
f_2, t_spec_2, Sxx_2 = signal.spectrogram(sig_changing, fs, nperseg=nperseg_2, noverlap=noverlap_2)

# Plot the spectrograms
plt.figure(figsize=(16, 6))

# Subplot 1: Spectrogram with window_length = 0.2s
plt.subplot(1, 2, 1)
plt.pcolormesh(t_spec_1, f_1, Sxx_1, shading='auto', cmap="jet")
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.ylim(0, 15)
plt.title('Spectrogram (window_length = 0.2s)')

# Subplot 2: Spectrogram with window_length = 1.0s
plt.subplot(1, 2, 2)
plt.pcolormesh(t_spec_2, f_2, Sxx_2, shading='auto', cmap="jet")
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.ylim(0, 15)
plt.title('Spectrogram (window_length = 1.0s)')

plt.tight_layout()
plt.show()

## III.3 The problem of Spectral Leakage
When we cut out a chunk of a signal, we create very sharp, artificial edges at the beginning and end. The Fourier Transform sees these sharp edges and gets confused. It thinks these abrupt changes are part of the signal itself.

This confusion causes spectral leakage. Instead of seeing a single, clean peak at the signal's true frequency (e.g., 8 Hz), the energy gets "smeared" or "leaks" into neighboring frequencies. It's like a blurry photo: you can still see the main subject, but it's not sharp, and it bleeds into the background. This makes it harder to pinpoint the exact frequencies and their true power.

In [ ]:
# FFT of the whole signal
yf_changing = rfft(sig_changing)
xf_changing = rfftfreq(len(sig_changing), 1/fs)

plt.figure(figsize=(10, 5))
plt.plot(xf_changing, np.abs(yf_changing))
plt.xlim(0, 10)
plt.title("FFT of the Entire Signal")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)
plt.show()

To solve this problem, we use a windowing function.

Think of it as a special filter that we apply to our signal chunk before we compute the FFT. This filter has a shape that is smooth, typically starting at zero, rising to one in the middle, and going back to zero at the end.

When we multiply our signal chunk by this window shape, it forces the chunk to start and end gently. The sharp, artificial edges are smoothed out.

In [ ]:
# --- 1. Create a signal with off-bin frequencies ---
fs = 1000  # Sampling rate
N = 1000   # Number of samples (1 second of data)
t = np.arange(N) / fs

# Frequency resolution is fs/N = 1 Hz.
# We choose frequencies that are not integers to maximize leakage.
f1 = 10.5  # Strong signal, halfway between 10 Hz and 11 Hz bins
f2 = 25.2  # Weaker signal, also off-bin
sig = 1.0 * np.sin(2 * np.pi * f1 * t) + 0.15 * np.sin(2 * np.pi * f2 * t)

# --- 2. Compute FFT with and without a window ---

# No window (equivalent to a rectangular window)
yf_no_window = rfft(sig)
xf = rfftfreq(N, 1 / fs)

# With a Hann window
hann_window = signal.get_window('hann', N)
sig_windowed = sig * hann_window
yf_windowed = rfft(sig_windowed)

# --- 3. Plot the results ---
plt.figure(figsize=(14, 10))

# Plot 1: The signal in the time domain
plt.subplot(2, 1, 1)
plt.plot(t, sig, label='Original Signal (Rectangular Window)')
plt.plot(t, sig_windowed, label='Signal with Hann Window', linewidth=2)
plt.title('Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)

# Plot 2: The FFTs in the frequency domain (log scale)
plt.subplot(2, 1, 2)
# Note: Using semilogy to see the leakage "floor"
plt.semilogy(xf, np.abs(yf_no_window), label='No Window (High Leakage)')
plt.semilogy(xf, np.abs(yf_windowed), label='With Hann Window (Low Leakage)', linewidth=2)
plt.title('FFT Comparison: The Effect of Windowing')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (log scale)')
plt.xlim(0, 40)
plt.ylim(1e-1, 1e3)
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Because the edges are no longer abrupt, the Fourier Transform doesn't get as confused. The spectral leakage is significantly reduced, and the energy in the FFT becomes much more concentrated around the true frequency. The result is a sharper, cleaner frequency peak, closer to the real signal content. The Hann window (hann in SciPy) is a very common and effective choice.

The problem now is that, since this windowing function is applied to every chunck of signal, that is to every single window, we are essentially ignoring the information at the very beginning and end of that chunk.

This is why we use *overlap*

## III.4 The Role of Overlap (`noverlap`)

**Overlap** solves this problem. We slide the next window back so that it covers the part of the signal that was tapered down in the previous window. The `noverlap` parameter specifies the number of points that are shared between consecutive windows.

We also use it to **increase time resolution:** Overlapping allows us to compute the FFT more frequently along the time axis of the signal. This results in more time points (columns) in the final spectrogram, providing a smoother and more detailed view of how the signal's frequency content evolves over time.

In [ ]:
# Create a signal with a short, transient event
fs = 1000
duration = 3.0
t = np.arange(0, duration, 1/fs)

# The transient event: a 15 Hz sine wave lasting 0.25 seconds
transient_duration = 0.25
t_transient = np.arange(0, transient_duration, 1/fs)
transient_signal = np.sin(2 * np.pi * 15 * t_transient)

# Create a baseline signal (e.g., zeros)
final_signal = np.zeros_like(t)

# --- Place the transient event right on the boundary of a 1-second window ---
# The first window ends at 1.0s, the second starts at 1.0s.
# We will center the 0.25s event at t=1.0s.
start_time = 1.0 - (transient_duration / 2) # Starts at 0.875s
start_index = int(start_time * fs)
end_index = start_index + len(transient_signal)
final_signal[start_index:end_index] = transient_signal

# --- Calculate Spectrograms ---
window_length = 1.0 # 1-second window
nperseg = int(window_length * fs)

# 1. No Overlap
f_0, t_0, Sxx_0 = signal.spectrogram(final_signal, fs, nperseg=nperseg, noverlap=0)

# 2. 50% Overlap
noverlap_50 = int(nperseg * 0.5)
f_50, t_50, Sxx_50 = signal.spectrogram(final_signal, fs, nperseg=nperseg, noverlap=noverlap_50)

In [ ]:
# --- Plotting ---
fig = plt.figure(figsize=(14, 10))

# Plot the original signal
ax0 = plt.subplot(3, 1, 1)
ax0.plot(t, final_signal)
ax0.set_title('Signal with a Transient Oscillation at t=1.0s')
ax0.set_xlabel('Time (s)')
ax0.set_ylabel('Amplitude')
# Mark the window boundaries for the no-overlap case
ax0.axvline(1.0, color='r', linestyle='--', label='Window Boundary')
ax0.axvline(2.0, color='r', linestyle='--')
ax0.legend()

# Plot spectrogram with no overlap
ax1 = plt.subplot(3, 1, 2, sharex=ax0)
ax1.pcolormesh(t_0, f_0, Sxx_0, shading='auto', cmap="jet")
ax1.set_title('No Overlap: Transient Event is Missed')
ax1.set_ylabel('Frequency [Hz]')
ax1.set_ylim(0, 25)

# Plot spectrogram with 50% overlap
ax2 = plt.subplot(3, 1, 3, sharex=ax0)
ax2.pcolormesh(t_50, f_50, Sxx_50, shading='auto', cmap="jet")
ax2.set_title('50% Overlap: Transient Event is Captured')
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(0, 25)

plt.tight_layout()
plt.show()

The most common choice and a perfect default is 50% overlap. For standard windows like the Hann window, a 50% overlap ensures that every data point in your signal is properly accounted for.

Use higher overlap (75% - 90%) when you have very short, transient events you want to capture. A higher overlap gives you more "looks" at the data, creating a smoother spectrogram with better time resolution, making it less likely you'll miss an event that falls on a window edge.

Avoid low overlap (< 50%): It's generally not recommended because you risk losing information from your signal.

## III.5 Example with real data

In [ ]:
window_length = 4.0 # seconds
overlapratio = 0.5  # 50% overlap
nperseg = int(window_length * fs_50hz)
noverlap = int(overlapratio * nperseg)
f, t_spec, Sxx = signal.spectrogram(traces_50hz, fs_50hz, nperseg=nperseg, noverlap=noverlap)

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 8))
gs = plt.GridSpec(2, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t_50hz, traces_50hz)
ax1.set_xlabel("")
ax1.set_xlim(0, duration_50hz)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx, shading='auto', cmap="jet")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

When working with real-world data, the power values in a spectrogram can vary significantly, often spanning several orders of magnitude. 

To address this, we typically convert the power values to a logarithmic scale, such as the decibel (dB) scale.

In [ ]:
Sxx_db = 10 * np.log10(Sxx + 1e-10)  # Convert to dB

# Plot
fig = plt.figure(figsize=(17, 8))
gs = plt.GridSpec(2, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t_50hz, traces_50hz)
ax1.set_xlabel("")
ax1.set_xlim(0, duration_50hz)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

Finally we can visualize how our notch filter suppressed the 50Hz noise at each time-point

In [ ]:
f, t_spec, Sxx_filtered = signal.spectrogram(filtered_50Hz, fs_50hz, nperseg=nperseg, noverlap=noverlap)
Sxx_filtered_db = 10 * np.log10(Sxx_filtered + 1e-10)  # Convert to dB

# Plot
fig = plt.figure(figsize=(17, 8))
gs = plt.GridSpec(2, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
im1 = ax1.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('')
ax1.set_title('Original signal')

# Colorbar
cax1 = fig.add_subplot(gs[0, 1])
fig.colorbar(im1, cax=cax1, label='Power')

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im2 = ax2.pcolormesh(t_spec, f, Sxx_filtered_db, shading='auto', cmap="jet")
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_title('Notch filtered signal')

# Colorbar
cax2 = fig.add_subplot(gs[1, 1])
fig.colorbar(im2, cax=cax2, label='Power')
